In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from fafbseg import flywire
import pymaid
import navis as nv
import numpy as np
import seaborn as sns
import scipy.stats as stats
import statsmodels
import scikit_posthocs as sp
import sys
import scipy
import random
import string
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.colors as colors

flywire.get_materialization_versions(dataset="production")

In [ ]:
#check available flywire versions
flywire.get_materialization_versions(dataset="production")
#connect your catmaid instance
catmaid_token = ""
instance=pymaid.CatmaidInstance('https://radagast.hms.harvard.edu/catmaidaedes', catmaid_token)

In [ ]:
def get_cable_overlap(OSNs, PN = None, volume = None):
    neurons = flywire.skeletonize_neuron(OSNs)
    if volume:
        neurons = nv.in_volume(neurons, volume)
    
    if PN == None:
        cable_overlap = nv.cable_overlap(neurons, neurons, dist= "2 microns")
        return cable_overlap
    
    else:
        PN = flywire.skeletonize_neuron(PN)
        PN_cable_overlap = nv.cable_overlap(neurons, PN, dist= "2 microns")
        return PN_cable_overlap
    
def get_cable_length(OSNs,volume = None):
    neurons = flywire.skeletonize_neuron(OSNs)
    if volume:
        neurons = nv.in_volume(neurons, volume)

    return neurons.cable_length

def get_total_synapses(OSNs, volume = None):
    neurons = flywire.skeletonize_neuron(OSNs)
    if volume:
        neurons = nv.in_volume(neurons, volume) 
    return flywire.get_synapses(neurons, pre=True, post=False, attach=True, dataset='production', min_score=65,progress=False)    

def get_num_connectors(neuron1, neuron2, volume = None):
    if volume:
        neuron1 = nv.in_volume(neuron1, volume) 
        neuron2 = nv.in_volume(neuron2, volume) 
    df = flywire.synapses.get_adjacency(neuron1, targets=neuron2, dataset='production', min_score=65, progress=True)
    return df.values[0][0]

def synapses_per_micron(cable_overlap_df, PN = None):
    cable_overlap_synapse_density = []
    for rowIndex, row in cable_overlap_df.iterrows(): #iterate over rows
        synapse_count = []
        cable_overlap_length = [] 
        for columnIndex, value in row.items():
            if value != 0:
                synapse_count.append(get_num_connectors(rowIndex, columnIndex))
                cable_overlap_length.append(value)
        cable_overlap_synapse_density.append(np.nanmean(np.array(synapse_count) / np.array(cable_overlap_length)) * 1000)   
    return pd.DataFrame(cable_overlap_synapse_density,columns=['overlap_density'], index=cable_overlap_df.index)

def get_neurons_without_overlap(cable_overlap_df):
    cable_overlap_df = cable_overlap_df.sum(axis=1)
    no_PN_overlap_neurons = [cable_overlap_df.index[index] for index, neuron in enumerate(cable_overlap_df) if neuron == 0.0]
    return no_PN_overlap_neurons

## Projection Neuron IDs for glomeruli
|Glomerulus|Right Hemisphere|Left Hemisphere|
|:---------|:------------:|-----------:|
|DA4l |720575940608970197 |720575940649045369|
|DC1| 720575940637056887|        720575940621529435|
|DL4  |720575940627708688      | 720575940617343316|
|DL5 |720575940617207185       | 720575940639080765|
|DM3| 720575940614727903       | 720575940630549370|
|DM4 |720575940623528925       | 720575940615366055|
|DP1m  |720575940618308825 |     720575940622726271|
|VA2| 720575940611079236       | 720575940609460491|
|VA4| 720575940617668747     |   720575940642086389|
|VA6 |720575940636873791 |       720575940637594334|
|VA7l | 720575940625784153  |    720575940618277184|
|VC1  |720575940637526190   |    720575940633190847|
|VC2  |720575940628739560 |      720575940630097103|
|VL2a  |720575940618295454   |   720575940603464672|
|VL2p |720575940628467611     |  720575940618865112|
|VM4  |720575940640971995|
|DA4m |720575940631930828|
|VM6 |720575940624057853|

## Glomeruli with more than one uPN projection neurons
|Glomerulus|Right Hemisphere|
|:---------|------------:|
|D |720575940603985952 720575940633804212 720575940627844734|
|DA1 |720575940614309535 720575940637208718 720575940613345442 720575940626034819 720575940619385765 720575940605102694 720575940603231916 720575940621185050|     
|DA2 |720575940622364184 720575940611849187  720575940626937617 720575940622734835|
|DC2 |720575940627160322 720575940616824588|
|DC3| 720575940630131663 720575940613419695 720575940634939327          720575940634715743|
|DL1 |720575940616079035 720575940622368792                             720575940613809554|
|DL2d |720575940628316047 720575940630928715 720575940620103590 720575940642046453|
|DL3 |720575940630778428 720575940621926669 720575940627353426 720575940630778428 720575940620638335|
|DM2 |720575940638633535 720575940630024566|
|DM5 |720575940644258327 720575940629926159 720575940640032896|
|DM6 |720575940631328104 720575940622785062 720575940620437339|
|DP1l |720575940648650884                                        720575940606762686|
|VA1d |720575940622811430 720575940640804853 720575940619620852|
|VA1v |720575940629097922 720575940620199962 720575940628283560 720575940629733626     720575940632720026 720575940634229615|
|VA3 |720575940656090017 720575940629009513 720575940604431584|
|VA5 |720575940619838912 720575940637009636 720575940637231593|
|VA7m |720575940620068518 720575940640398477 720575940639680957|
|VC3m |720575940608845525 720575940608199748 720575940624515571|
|VL1 |720575940624863015 720575940637712243 720575940620595053|
|VM1 |720575940628077568  720575940623444715|
|VM2 |720575940640697808 720575940617179451|
|VM3 |720575940627480586 720575940610048981|
|VM7d |720575940628524292 720575940613167007|
|VM7v |720575940619337536 720575940621256212|
|VM5d |720575940611372714 720575940644961588 720575940635082871 720575940627035388|


In [ ]:
DA4l_pn = 720575940608970197
DC1_pn = 720575940637056887
DL4_pn = 720575940627708688
DL5_pn = [720575940617207185, 720575940630432815]
DM3_pn = 720575940614727903
DM4_pn = [720575940623528925 , 720575940628323919, 720575940606767806]	 
DP1m_pn = 720575940618308825
VA2_pn = 720575940611079236
VA4_pn = 720575940617668747
VA6_pn = 720575940636873791
VA7l_pn = 720575940625784153
VC1_pn = 720575940637526190
VC2_pn = 720575940628739560
VL2a_pn = 720575940618295454	
VL2p_pn = 720575940628467611
VM4_pn = 720575940640971995	
DM1_pn = 720575940630770042
V_bilateral_rightsoma_pn=720575940626143806
V_bilateral_leftsoma_pn=720575940630546540
V_unilateral_pn=720575940637910106
V_pn = [720575940626143806,720575940630546540,720575940637910106]
D_pn = [720575940603985952, 720575940633804212, 720575940627844734]
DA1_pn = [720575940614309535, 720575940637208718, 720575940613345442, 720575940626034819, 720575940619385765, 720575940605102694, 720575940603231916, 720575940621185050,720575940646122804] 
DA2_pn = [720575940622364184, 720575940611849187, 720575940626937617, 720575940638719104,720575940622734835]
DA4m_pn = 720575940631930828
DC2_pn = [720575940627160322, 720575940616824588]
DC3_pn = [720575940630131663, 720575940613419695, 720575940634939327] 
DL1_pn = [720575940616079035, 720575940622368792] 
DL2d_pn = [720575940628316047, 720575940630928715, 720575940620103590, 720575940642046453,720575940605421442, 720575940613706866, 720575940623922869]
DL3_pn = [720575940630778428, 720575940621926669, 720575940627353426, 720575940630778428, 720575940620638335]
DM2_pn = [720575940638633535, 720575940630024566]
DM5_pn = [720575940644258327, 720575940629926159, 720575940640032896]
DM6_pn = [720575940631328104, 720575940622785062, 720575940620437339]
DP1l_pn = [720575940648650884, 720575940606762686]
VA1d_pn = [720575940622811430, 720575940640804853, 720575940619620852]
VA1v_pn = [720575940629097922, 720575940620199962, 720575940628283560, 720575940629733626, 720575940632720026, 720575940634229615]
VA3_pn = [720575940656090017,  720575940604431584]
VA5_pn = [720575940619838912, 720575940637009636, 720575940637231593]
VA7m_pn = [720575940620068518, 720575940640398477, 720575940639680957]
VC5_pn = [720575940608845525, 720575940608199748, 720575940624515571] #VC3m PNs due to naming changing VC3m -> VC5
VL1_pn	= [720575940624863015,  720575940620595053]
VM1_pn = [720575940628077568,  720575940623444715]
VM2_pn = [720575940640697808, 720575940617179451]
VM3_pn = [720575940627480586, 720575940610048981]
VM6_pn = 720575940624057853
VM7d_pn = [720575940628524292, 720575940613167007]
VM7v_pn = [720575940619337536, 720575940621256212]
VM5d_pn = [720575940611372714, 720575940644961588, 720575940635082871, 720575940627035388]

# added from 2020 Current Biology
DA3_pn = [720575940625924618, 720575940659400577]
DC4_pn = 720575940613579943
DL2v_pn = [720575940620467438,720575940611022515,720575940628181520	,720575940627772009]
VC3_pn = [720575940615394719,720575940619902598	,720575940619928429	,720575940620465904]
VC4_pn = [720575940624001321,720575940635933119,720575940609454603]
VM5v_pn = [720575940610505170	,720575940620189790	,720575940625431866	]

In [ ]:
#load .ply files

dm1_l=nv.read_mesh("../glom_meshes/DM1_R.ply", output='volume') # DM1
v_l=nv.read_mesh("../glom_meshes/V_R.ply", output='volume') # V

gloms = ["DA4l", "DC1", "DL4", "DL5", "DM3","DM4", "DP1m", 
"VA2", "VA4", "VA6", "VA7l", "VC1", "VC2", "VL2a",
"VL2p", "VM4","DM1","V",
"D", 
"DA1",  
"DA2",  
"DA4m",  
"DC2", 
"DC3",  
"DL1", 
"DL2d",  
"DL3",  
"DM2",  
"DM5",  
"DM6",  
"DP1l", 
"VA1d",  
"VA1v",  
"VA3",  
"VA5", 
"VA7m", 
"VL1", 	
"VM1",  
"VM2",  
"VM3",  
"VM7d",  
"VM7v",  
"VM5d",
"DA3",
"DC4",
"DL2v",
"VC3",
'VC5',
'VM6',
"VC4",
"VM5v",
]

for glom in gloms:
    print(glom)
    exec(glom +"_mesh = nv.read_mesh(" + "'../glom_meshes/" + glom + "_R.ply' , output='volume')")

In [ ]:
#retrieve neuron_ids
glomeruli_list = ["ORN_DA4l",
                  "ORN_DC1", 
                  "ORN_DL4", 
                  "ORN_DL5",
                  "ORN_DM3", 
                  "ORN_DM4", 
                  "ORN_DP1m", 
                  "ORN_VA2", 
                  "ORN_VA4",
                  "ORN_VA6", 
                  "ORN_VA7l", 
                  "ORN_VC1", 
                  "ORN_VC2", 
                  "ORN_VL2a",
                  "ORN_VL2p", 
                  "ORN_VM4",
                  "ORN_DM1",
                  "ORN_V",
                  "ORN_D", 
                  "ORN_DA1",  
                  "ORN_DA2",  
                  "ORN_DA4m",  
                  "ORN_DC2", 
                  "ORN_DC3",  
                  "ORN_DL1", 
                  "ORN_DL2d",  
                  "ORN_DL3",  
                  "ORN_DM2",  
                  "ORN_DM5",  
                  "ORN_DM6",  
                  "ORN_DP1l", 
                  "ORN_VA1d",  
                  "ORN_VA1v",  
                  "ORN_VA3",  
                  "ORN_VA5", 
                  "ORN_VA7m", 
                  "ORN_VC3",
                  'ORN_VC5',
                  'ORN_VM6',  
                  "ORN_VL1", 	
                  "ORN_VM1",  
                  "ORN_VM2",  
                  "ORN_VM3",  
                  "ORN_VM7d",  
                  "ORN_VM7v",  
                  "ORN_VM5d",
                  "ORN_DA3",
                  "ORN_DC4",
                  "ORN_DL2v",
                  "ORN_VC3l",
                  "ORN_VC4",
                  "ORN_VM5v",
]  
                  
glomeruli_OSNs = pd.DataFrame()
for glomeruli in glomeruli_list:
    if glomeruli in ['ORN_VC3','ORN_VC5','ORN_VM6']:
        annotation_search = flywire.search_annotations(flywire.NeuronCriteria(cell_type=glomeruli, side='left', annotation_version="v2.0.0"))
    else:
        print(glomeruli)
        annotation_search = flywire.search_annotations(flywire.NeuronCriteria(hemibrain_type=glomeruli, side='left', annotation_version="v2.0.0"))
    print(f' for {glomeruli} {annotation_search.shape[0]} neurons were found')
    glomeruli_OSNs = pd.concat([glomeruli_OSNs, annotation_search])
    glomeruli_OSNs = glomeruli_OSNs[['root_id','hemibrain_type','cell_type']] #keep just the root_id and the glomeruli

In [ ]:
#the following neurons have no synapses with projection neurons in their glomerulus, so they were excluded from all analysis
neurons_to_exclude = [720575940642936520, 720575940623359466, 720575940633033133, 720575940621862605,
                      720575940605724081, 720575940604876977, 720575940630971434, 720575940632824531,
                      720575940611791834, 720575940614890642, 720575940611437973, 720575940619969329, 	
                      720575940614019112, 720575940615639199, 720575940639118653, 720575940615629173,
                      720575940621082138, 720575940608615363, 720575940630679749, 720575940628398791,
                      720575940639089571, 720575940634883179, 720575940638202852, 720575940614006934,
                      720575940603870688, 720575940632033875, 720575940632391736, 720575940623750852,
                      720575940622248709, 720575940631930415, 720575940613885506, 720575940627787343,
                      720575940624671490, 720575940631393823, 720575940619029589, 720575940623655605,
                      720575940630462572, 
                    ]

In [ ]:
glomeruli_OSNs = glomeruli_OSNs[~glomeruli_OSNs['root_id'].isin(neurons_to_exclude)]

In [ ]:
glom_list = ["D", 
"DA1", 
"DA2", 
"DA3",
"DA4l", 
"DA4m", 
"DC1",
"DC2", 
"DC3", 
"DC4",
"DL1", 
"DL2d", 
"DL2v",
"DL3", 
"DL4",
"DL5",
"DM1",
"DM2", 
"DM3",
"DM4",
"DM5", 
"DM6", 
"DP1l", 
"DP1m",
"V",
"VA1d",
"VA1v",
"VA2", 
"VA3", 
"VA4",
"VA5", 
"VA6",
"VA7l",
"VA7m", 
"VC1",
"VC2", 
"VC3",
"VC4",
"VC5",
"VL1",
"VL2a",
"VL2p",
"VM1",
"VM2",
"VM3", 
"VM4",
"VM5d"
"VM5v",  
"VM6",
"VM7d", 
"VM7v"]



for glom in glom_list:
   retries = 5
   for retry in range(retries):
        try:
            print(glom)
            if glom in ['VM6']:
                exec(f"{glom}_feedforward_synapses = flywire.get_adjacency(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'], {glom}_pn, min_score=65, dataset='production')")#.sum(axis=1)")
                exec(f"{glom}_recurrent_synapses = flywire.get_adjacency(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'], min_score=65, dataset='production')")#.sum(axis=1)")
                exec(f"{glom}_total_synapses = get_total_synapses(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'], volume = {glom}_mesh)")
                exec(f"{glom}_feedforward = get_cable_overlap(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'],PN={glom}_pn, volume={glom}_mesh)")
                exec(f"{glom}_recurrent = get_cable_overlap(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'],volume={glom}_mesh)")          
                exec(f"{glom}_cable_length = get_cable_length(glomeruli_OSNs[(glomeruli_OSNs['cell_type']=='ORN_{glom}v') | (glomeruli_OSNs['cell_type']=='ORN_{glom}l') | (glomeruli_OSNs['cell_type']=='ORN_{glom}m')]['root_id'], volume={glom}_mesh)")
            elif glom in ['VC3','VC5']: 
                exec(f"{glom}_feedforward_synapses = flywire.get_adjacency(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'], {glom}_pn, min_score=65, dataset='production')")#.sum(axis=1)")
                exec(f"{glom}_recurrent_synapses = flywire.get_adjacency(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'], min_score=65, dataset='production')")#.sum(axis=1)")
                exec(f"{glom}_total_synapses = get_total_synapses(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'], volume = {glom}_mesh)")
                exec(f"{glom}_feedforward = get_cable_overlap(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'],PN={glom}_pn, volume={glom}_mesh)")
                exec(f"{glom}_recurrent = get_cable_overlap(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'],volume={glom}_mesh)")
                exec(f"{glom}_cable_length = get_cable_length(glomeruli_OSNs[glomeruli_OSNs['cell_type']=='ORN_{glom}']['root_id'], volume={glom}_mesh)") 
            else:
              exec(f"{glom}_feedforward_synapses = flywire.get_adjacency(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'], {glom}_pn, min_score=65, dataset='production')")#.sum(axis=1)")
              exec(f"{glom}_recurrent_synapses = flywire.get_adjacency(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'], min_score=65, dataset='production')")#.sum(axis=1)")
              exec(f"{glom}_total_synapses = get_total_synapses(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'], volume = {glom}_mesh)")
              exec(f"{glom}_feedforward = get_cable_overlap(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'],PN={glom}_pn, volume={glom}_mesh)")
              exec(f"{glom}_recurrent = get_cable_overlap(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'],volume={glom}_mesh)")
              exec(f"{glom}_cable_length = get_cable_length(glomeruli_OSNs[glomeruli_OSNs['hemibrain_type']=='ORN_{glom}']['root_id'], volume={glom}_mesh)")
            exec(f"{glom}_feedforward_synapses_per_micron = synapses_per_micron({glom}_feedforward,PN={glom}_pn)")
            exec(f"{glom}_recurrent_synapses_per_micron = synapses_per_micron({glom}_recurrent)")
        except:
            if retry < retries - 1:
                print(f'number of retries left = {retries-retry}')
                continue
            else:
                raise
        break

retrieve mosquito data from catmaid

In [ ]:
#MD1
MD1OSNs = pymaid.get_skids_by_annotation(['innervates MD1', 'left palp nerve', 'sensory neuron', 'PSPs done'], allow_partial = False, intersect = True)
MD1OSNsALL = pymaid.get_skids_by_annotation(['innervates MD1', 'left palp nerve', 'sensory neuron'], allow_partial = False, intersect = True)
MD1volume = pymaid.get_volume('MD1 04/06/21')
MD1neurons = pymaid.get_neuron(MD1OSNs)
MD1neurons = pymaid.CatmaidNeuron.prune_by_volume(MD1neurons,v=MD1volume,mode='IN',prevent_fragments=True)
MD1neuronsALL = pymaid.get_neuron(MD1OSNsALL)

In [ ]:

#MD2
MD2OSNs = pymaid.get_skids_by_annotation(['innervates MD2', 'left palp nerve', 'sensory neuron', 'PSPs done'], allow_partial = False, intersect = True)
MD2OSNsALL = pymaid.get_skids_by_annotation(['innervates MD2', 'left palp nerve', 'sensory neuron'], allow_partial = False, intersect = True)
MD2volume = pymaid.get_volume('MD2 2022')
MD2neurons = pymaid.get_neuron(MD2OSNs)
MD2neurons = pymaid.CatmaidNeuron.prune_by_volume(MD2neurons,v=MD2volume,mode='IN',prevent_fragments=True)
MD2neuronsALL = pymaid.get_neuron(MD2OSNsALL)



In [ ]:
#MD3
MD3OSNs = pymaid.get_skids_by_annotation(['innervates MD3', 'left palp nerve', 'sensory neuron', 'PSPs done'], allow_partial = False, intersect = True)
MD3OSNsALL = pymaid.get_skids_by_annotation(['innervates MD3', 'left palp nerve', 'sensory neuron'], allow_partial = False, intersect = True)
MD3volume = pymaid.get_volume('MD3 2022')
MD3neurons = pymaid.get_neuron(MD3OSNs)
MD3neurons = pymaid.CatmaidNeuron.prune_by_volume(MD3neurons,v=MD3volume,mode='IN',prevent_fragments=True)
MD3neuronsALL = pymaid.get_neuron(MD3OSNsALL)



In [ ]:
#initialize empty dataframe
flyneurons=pd.DataFrame(columns=['id', 'total cable length', 'cable length in glom', 'unilateral feedforward', 'bilateral left feedforward', 'bilateral right feedforward', 'recurrent connections', 'glomerulus'])

md1_synapse_count = []
md1_synapse_count_in_volume = []
md1_total_synapses = []
md1_total_synapses_restricted = []
md1_synapse_count_feedforward = []
md1_synapse_count_feedforward_unrestricted = []
md1_cable_overlap = []
md1_feedforward_density = []
md1_feedforward_density_unrestricted = []
md1_recurrent_density = []
md1_recurrent_density_unrestricted = []
md1_cable_length = []
md1_cable_overlap_percentage = []
md1_feedforward_fraction = []
md1_recurrent_fraction = []
md1_recurrent_fraction_restricted = []
md1_recurrent_synapses = []
md1_recurrent_synapses_restricted = []

for n in MD1neurons:
    skeletonid=n.skeleton_id
    skelid=int(skeletonid)
    print(skelid)
    labels = pymaid.get_label_list()
    neuronlabels=labels[labels.skeleton_id==skelid]
    branchpoint=neuronlabels[neuronlabels.tag=='first branch point']

    if len(branchpoint) !=1:
        print('error, neuron skelid=%i does not have exactly 1 branchpoint tag'% skelid)
    else: bpnode=branchpoint.node_id.values[0]

    PN = pymaid.get_neuron('295')

    cable_overlap_length = nv.cable_overlap(n, PN, dist= "2 microns")

    connections = pymaid.get_partners(n).set_index('skeleton_id')
    connections = connections[connections['relation'] == 'downstream']
    total_synapses = pd.DataFrame(connections['total']).transpose()
    total_synapses.columns.names = ['targets']
    total_synapses.index = [n.id]
    total_synapses.index.names = ['sources']
    total_synapses_restricted = pymaid.filter_connectivity(total_synapses, restrict_to='MD1 04/06/21')
    
    #feedforward connections
    feedforward = pymaid.adjacency_matrix(n, targets=295)
    glom_connectivity=pymaid.filter_connectivity(feedforward, restrict_to='MD1 04/06/21')
    feedforward_value=int(glom_connectivity.iloc[0])

    #recurrent connections
    recurrent = pymaid.adjacency_matrix(n, targets=MD1neuronsALL)
    recurrent_glom_connectivity=pymaid.filter_connectivity(recurrent, restrict_to='MD1 04/06/21')
    print(recurrent_glom_connectivity)

    cable_overlap = nv.cable_overlap(n,MD1neuronsALL,dist= "2 microns")
    synapse_density_unrestricted = recurrent / (cable_overlap.values / 1000)
    synapse_density = recurrent_glom_connectivity / (cable_overlap.values / 1000)

    md1_total_synapses.append(sum(total_synapses.iloc[0]))
    md1_total_synapses_restricted.append(sum(total_synapses_restricted.iloc[0]))
    md1_synapse_count.append(recurrent.values)
    md1_synapse_count_feedforward.append(feedforward_value)
    md1_synapse_count_feedforward_unrestricted.append(int(feedforward.iloc[0]))
    md1_cable_overlap.append(cable_overlap.values)
    md1_feedforward_density.append((glom_connectivity/(cable_overlap_length/1000)).iloc[0].values[0])
    md1_feedforward_density_unrestricted.append((int(feedforward.iloc[0])/(cable_overlap_length/1000)).iloc[0].values[0])
    md1_recurrent_density.append(synapse_density.mean(axis=1).iloc[0])
    md1_recurrent_density_unrestricted.append(synapse_density_unrestricted.mean(axis=1).iloc[0])
    md1_cable_length.append(n.cable_length/1000)
    md1_cable_overlap_percentage.append(((cable_overlap.values /1000)/(n.cable_length/1000)).mean())
    sum_recurrent=recurrent.iloc[0].sum()
    md1_feedforward_fraction.append(feedforward_value / total_synapses_restricted.sum(axis=1)[0])
    md1_recurrent_synapses.append(recurrent.iloc[0].sum())
    md1_recurrent_synapses_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0])
    md1_recurrent_fraction.append(sum_recurrent / total_synapses)
    md1_recurrent_fraction_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0]/sum(total_synapses_restricted.iloc[0]))


    #append to dataframe
    flyneurons= pd.concat([flyneurons, 
                           pd.DataFrame({'id':n,
                                         'total cable length':n.cable_length,
                                         'cable length in glom': cable_overlap_length.values[0][0], #dlength, 
                                         'unilateral feedforward':feedforward_value,
                                         'recurrent connections': sum_recurrent,
                                         'glomerulus':'Glomerulus 1'}, index = [0])
                                         ],
                           ignore_index=True)

In [ ]:
#initialize empty dataframe
flyneurons=pd.DataFrame(columns=['id', 'total cable length', 'cable length in glom', 'unilateral feedforward', 'bilateral left feedforward', 'bilateral right feedforward', 'recurrent connections', 'glomerulus'])

md2_synapse_count = []
md2_synapse_count_in_volume = []
md2_total_synapses = []
md2_total_synapses_restricted = []
md2_synapse_count_feedforward = []
md2_synapse_count_feedforward_unrestricted = []
md2_cable_overlap = []
md2_feedforward_density = []
md2_feedforward_density_unrestricted = []
md2_recurrent_density = []
md2_recurrent_density_unrestricted = []
md2_cable_length = []
md2_cable_overlap_percentage = []
md2_feedforward_fraction = []
md2_recurrent_fraction = []
md2_recurrent_fraction_restricted = []
md2_recurrent_synapses = []
md2_recurrent_synapses_restricted = []

for n in MD2neurons:
    skeletonid=n.skeleton_id
    skelid=int(skeletonid)
    print(skelid)
    labels = pymaid.get_label_list()
    neuronlabels=labels[labels.skeleton_id==skelid]
    branchpoint=neuronlabels[neuronlabels.tag=='first branch point']

    if len(branchpoint) !=1:
        print('error, neuron skelid=%i does not have exactly 1 branchpoint tag'% skelid)
    else: bpnode=branchpoint.node_id.values[0]

    PN = pymaid.get_neuron('690')

    cable_overlap_length = nv.cable_overlap(n, PN, dist= "2 microns")

    connections = pymaid.get_partners(n).set_index('skeleton_id')
    connections = connections[connections['relation'] == 'downstream']
    total_synapses = pd.DataFrame(connections['total']).transpose()
    total_synapses.columns.names = ['targets']
    total_synapses.index = [n.id]
    total_synapses.index.names = ['sources']
    total_synapses_restricted = pymaid.filter_connectivity(total_synapses, restrict_to='MD2 2022')
    
    #feedforward connections
    feedforward = pymaid.adjacency_matrix(n, targets=690)
    glom_connectivity=pymaid.filter_connectivity(feedforward, restrict_to='MD2 2022')
    feedforward_value=int(glom_connectivity.iloc[0])

    #recurrent connections
    recurrent = pymaid.adjacency_matrix(n, targets=MD2neuronsALL)
    recurrent_glom_connectivity=pymaid.filter_connectivity(recurrent, restrict_to='MD2 2022')
    print(recurrent_glom_connectivity)

    cable_overlap = nv.cable_overlap(n,MD2neuronsALL,dist= "2 microns")
    synapse_density_unrestricted = recurrent / (cable_overlap.values / 1000)
    synapse_density = recurrent_glom_connectivity / (cable_overlap.values / 1000)

    md2_total_synapses.append(sum(total_synapses.iloc[0]))
    md2_total_synapses_restricted.append(sum(total_synapses_restricted.iloc[0]))
    md2_synapse_count.append(recurrent.values)
    md2_synapse_count_feedforward.append(feedforward_value)
    md2_synapse_count_feedforward_unrestricted.append(int(feedforward.iloc[0]))
    md2_cable_overlap.append(cable_overlap.values)
    md2_feedforward_density.append((glom_connectivity/(cable_overlap_length/1000)).iloc[0].values[0])
    md2_feedforward_density_unrestricted.append((int(feedforward.iloc[0])/(cable_overlap_length/1000)).iloc[0].values[0])
    md2_recurrent_density.append(synapse_density.mean(axis=1).iloc[0])
    md2_recurrent_density_unrestricted.append(synapse_density_unrestricted.mean(axis=1).iloc[0])
    md2_cable_length.append(n.cable_length/1000)
    md2_cable_overlap_percentage.append(((cable_overlap.values /1000)/(n.cable_length/1000)).mean())
    sum_recurrent=recurrent.iloc[0].sum()
    md2_feedforward_fraction.append(feedforward_value / total_synapses_restricted.sum(axis=1)[0])
    md2_recurrent_synapses.append(recurrent.iloc[0].sum())
    md2_recurrent_synapses_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0])
    md2_recurrent_fraction.append(sum_recurrent / total_synapses)
    md2_recurrent_fraction_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0]/sum(total_synapses_restricted.iloc[0]))


    #append to dataframe
    flyneurons= pd.concat([flyneurons, 
                           pd.DataFrame({'id':n,
                                         'total cable length':n.cable_length,
                                         'cable length in glom': cable_overlap_length.values[0][0], #dlength, 
                                         'unilateral feedforward':feedforward_value,
                                         'recurrent connections': sum_recurrent,
                                         'glomerulus':'Glomerulus 1'}, index = [0])
                                         ],
                           ignore_index=True)

In [ ]:
#initialize empty dataframe
flyneurons=pd.DataFrame(columns=['id', 'total cable length', 'cable length in glom', 'unilateral feedforward', 'bilateral left feedforward', 'bilateral right feedforward', 'recurrent connections', 'glomerulus'])

md3_synapse_count = []
md3_synapse_count_in_volume = []
md3_total_synapses = []
md3_total_synapses_restricted = []
md3_synapse_count_feedforward = []
md3_synapse_count_feedforward_unrestricted = []
md3_cable_overlap = []
md3_feedforward_density = []
md3_feedforward_density_unrestricted = []
md3_recurrent_density = []
md3_recurrent_density_unrestricted = []
md3_cable_length = []
md3_cable_overlap_percentage = []
md3_feedforward_fraction = []
md3_recurrent_fraction = []
md3_recurrent_fraction_restricted = []
md3_recurrent_synapses = []
md3_recurrent_synapses_restricted = []

for n in MD3neurons:
    skeletonid=n.skeleton_id
    skelid=int(skeletonid)
    print(skelid)
    labels = pymaid.get_label_list()
    neuronlabels=labels[labels.skeleton_id==skelid]
    branchpoint=neuronlabels[neuronlabels.tag=='first branch point']

    if len(branchpoint) !=1:
        print('error, neuron skelid=%i does not have exactly 1 branchpoint tag'% skelid)
    else: bpnode=branchpoint.node_id.values[0]

    PN = pymaid.get_neuron('11126')

    cable_overlap_length = nv.cable_overlap(n, PN, dist= "2 microns")

    connections = pymaid.get_partners(n).set_index('skeleton_id')
    connections = connections[connections['relation'] == 'downstream']
    total_synapses = pd.DataFrame(connections['total']).transpose()
    total_synapses.columns.names = ['targets']
    total_synapses.index = [n.id]
    total_synapses.index.names = ['sources']
    total_synapses_restricted = pymaid.filter_connectivity(total_synapses, restrict_to='MD3 2022')
    
    #feedforward connections
    feedforward = pymaid.adjacency_matrix(n, targets=11126)
    glom_connectivity=pymaid.filter_connectivity(feedforward, restrict_to='MD3 2022')
    feedforward_value=int(glom_connectivity.iloc[0])

    #recurrent connections
    recurrent = pymaid.adjacency_matrix(n, targets=MD3neuronsALL)
    recurrent_glom_connectivity=pymaid.filter_connectivity(recurrent, restrict_to='MD3 2022')
    print(recurrent_glom_connectivity)

    cable_overlap = nv.cable_overlap(n,MD3neuronsALL,dist= "2 microns")
    synapse_density_unrestricted = recurrent / (cable_overlap.values / 1000)
    synapse_density = recurrent_glom_connectivity / (cable_overlap.values / 1000)

    md3_total_synapses.append(sum(total_synapses.iloc[0]))
    md3_total_synapses_restricted.append(sum(total_synapses_restricted.iloc[0]))
    md3_synapse_count.append(recurrent.values)
    md3_synapse_count_feedforward.append(feedforward_value)
    md3_synapse_count_feedforward_unrestricted.append(int(feedforward.iloc[0]))
    md3_cable_overlap.append(cable_overlap.values)
    md3_feedforward_density.append((glom_connectivity/(cable_overlap_length/1000)).iloc[0].values[0])
    md3_feedforward_density_unrestricted.append((int(feedforward.iloc[0])/(cable_overlap_length/1000)).iloc[0].values[0])
    md3_recurrent_density.append(synapse_density.mean(axis=1).iloc[0])
    md3_recurrent_density_unrestricted.append(synapse_density_unrestricted.mean(axis=1).iloc[0])
    md3_cable_length.append(n.cable_length/1000)
    md3_cable_overlap_percentage.append(((cable_overlap.values /1000)/(n.cable_length/1000)).mean())
    sum_recurrent=recurrent.iloc[0].sum()
    md3_feedforward_fraction.append(feedforward_value / total_synapses_restricted.sum(axis=1)[0])
    md3_recurrent_synapses.append(recurrent.iloc[0].sum())
    md3_recurrent_synapses_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0])
    md3_recurrent_fraction.append(sum_recurrent / total_synapses)
    md3_recurrent_fraction_restricted.append(recurrent_glom_connectivity.sum(axis=1)[0]/sum(total_synapses_restricted.iloc[0]))


    #append to dataframe
    flyneurons= pd.concat([flyneurons, 
                           pd.DataFrame({'id':n,
                                         'total cable length':n.cable_length,
                                         'cable length in glom': cable_overlap_length.values[0][0], #dlength, 
                                         'unilateral feedforward':feedforward_value,
                                         'recurrent connections': sum_recurrent,
                                         'glomerulus':'Glomerulus 1'}, index = [0])
                                         ],
                           ignore_index=True)